This notebook contains the main parameters and simulation data to solve the single track fusion of 316L metal.

It contains executable Python code that is available at [https://github.com/pzimbrod/galerkin-pbf-lb/blob/main/powder-bed-fusion/main.ipynb](https://github.com/pzimbrod/galerkin-pbf-lb/blob/main/powder-bed-fusion/main.ipynb).
To run it, follow the instructions in the [Readme](https://github.com/pzimbrod/galerkin-pbf-lb/tree/main/powder-bed-fusion) file on GitHub.

For the finite element implementation, the library `FEniCSx` is used.

The core data structures and methods that are needed in order to describe and assemble this model are collected in the class `PBFModel` in this directory.

In [ ]:
from PBFModel.geometry import check_msh_file
import os
from PBFModel.PBFModel import PBFModel
from dolfinx import log

# Geometry

The computational domain is created and triangulated using the `gmsh` software package.

Here, a `.geo` file created by `gmsh` is read in, where the parameters of the three-dimensional, box-shaped domain are defined.

To define the boundary conditions properly, the exterior facets of the domain have to be properly subdivided and assigned unique identifiers.
This is also done in the input file. The `markers` dictionary reflects these markers which have integer values.
Using these markers, the facets that are assigned to a specidic subdomain of the boundary can be accessed, e.g. the pre-heated, solidified bottom of the domain.

In [ ]:
#========   Set this to True if you want to overwrite an existing mesh (if one is present) ==========#
# BUG: Topology computation for hex mesh never finishes, use tets for now
use_hex_mesh = False
create_new_mesh = False

current_directory = os.getcwd()
mesh_already_present = check_msh_file(current_directory)
model_rank = 0

# from GEO file
markers = {
    "inlet":    23,
    "outlet":   24,
    "walls":    25,
    "bottom":   21
}
if not mesh_already_present or create_new_mesh:
    os.system("gmsh -3 -format msh2 lpbf/mesh3d.geo -o mesh3d.msh")

# Finite Element Configuration

One can either create the problem in a mixed formulation or in a separated fashion.

In the former variant, the solution to the problem becomes one global vector of all unknowns of the model, in this case:

$$ \underline{u} = \begin{bmatrix} \alpha_s \\ \alpha_l \\ \alpha_g \\ p \\ u \\ T \end{bmatrix} $$

This creates a global nonlinear problem, which is solved at once in a monolithic fashion, which may for complicated problems be tedious to solve and slow to converge.
However, relatively few steps are required in order to set up the actual numerical solver.

Alternatively, one can create the variables of the PDE separately, i.e. in separate function spaces.
In this case, the problem are also solved in separate stages, possibly yielding large speedups in the solution process.
As a downside, however, this approach requires intricate knowledge about efficiently solving each system, for example pressure predictor-corrector schemes for the Navier-Stokes equations.

For sake of simplicity, a mixed problem is created in this case:

In [ ]:
create_mixed_problem = True

To set up the finite elements and function spaces, some choices have to be made regarding the types and degrees of the elements.

Details about the element choice can be found in the dissertation thesis.
In principle, one must choose between `"CG"` and `"DG"` approximations, that is, function spaces using either continuous or discontinuous Lagrange polynomials, respectively.

In addition, for each field, a time stepping scheme is required.
In this case, one can choose between the explicit euler
$$
u_{n+1} = u_n + G(u_n)
$$
or implicit euler scheme
$$
u_{n+1} = u_n + G(u_{n+1}),
$$
where $G(u)$ contains the spatial differential operators of a PDE.

In [ ]:
fe_config = {

    "alpha_solid": {
        "element": "DG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },

    "alpha_liquid": {
        "element": "DG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },

    "alpha_gas": {
        "element": "DG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },

    "p": {
        "element": "CG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },

    "u": {
        "element": "CG", 
        "degree": 2, 
        "type": "vector",
        "time_scheme":  "explicit euler",
        },

    "T": {
        "element": "CG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "implicit euler",
        },
}

# Material Model

Next, the thermophysical properties of the simulation are defined per phase.

The solid, liquid and gaseous phase each need to be assigned a density (`rho`), thermal conductivity (`cp`) and thermal conductivity (`kappa`).

In [ ]:
# This should be a dictionary of dictionaries
material_model = {
    "solid": {
        "rho":      7800.0, # kg/m3
        "cp":       502.4,  # J/(kg K)
        "kappa":    14.4,   # W/(m K)
    },
    "liquid": {
        "rho":      7800.0, # kg/m3
        "cp":       502.4,  # J/(kg K)
        "kappa":    14.4,   # W/(m K)
    },
    "gas": {
        "rho":      7800.0, # kg/m3
        "cp":       502.4,  # J/(kg K)
        "kappa":    14.4,   # W/(m K)
    },
}

# Time Domain

The time span to be simulated is passed in the tuple `time_domain`. The time step is assumed to be constant, and in case of an explicit time discretisation must be chosen small enough such that the simulation remains stable.

For Discontinuous Galerkin methods, a modified version of the Courant-Friedrichs-Lewy (CFL) condition holds
$$
\Delta t \leq \frac{h}{u_{\mathrm{max}}} \frac{1}{2p+1},
$$
where $h$ is the (minimum) grid size, $u_\mathrm{max}$ is the maximum flow velocity in the model or an estimate thereof and $p$ is the order of the Finite Element.

Given a shielding gas flow velocity of 1 m/s, a discretisation size of $1e-6$ m and a polynomial order of $p=1$, the allowable time step becomes $\Delta t \leq 5e-7$.

In [ ]:
time_domain = (0.0, 2e-3)

def compute_dt(h: float, u_max: float, p: int) -> float:
    return 0.8 * h/u_max * 1/(2*p + 1)

dt = compute_dt(h=1e-6, u_max=1.0, p=1)
print(dt)

# Top Level Data Structure

All actual data is contained in the `PBFModel` class.

Using the input data above, the constructor for this class can be called.
Then, the problem class is instantiated, creating all necessary function spaces, functions and constants in order to solve the problem.
A central element is the `FEData` subclass. It is designed to contain all Finite Element specific aspects of the model.

In [ ]:
#log.set_log_level(log.LogLevel.INFO)
model = PBFModel(mesh_path="mesh3d.msh",
                 fe_config=fe_config,
                 material_model=material_model,
                 bc_markers=markers, 
                 timestep=dt,
                 time_domain=time_domain,
                 create_mixed=create_mixed_problem)

After creation, calling the `setup()` method assigns the appropriate initial and boundary conditions that are defined within the class. They can be found in the files `InitialCondition.py` and `BoundaryCondition.py`.

In [ ]:
model.setup()

The model can be executed using the `solve()` method.
Then, the simulation is run over all time steps. Note that this can take a considerable amount of time.

In [ ]:
model.solve()